In [1]:
import numpy as np
import pandas as pd
import math 

params = {
    'pi' : [np.pi],
    'alpha' : [0.1],
    'alpha_gc' : [50],
    'a' : [3],
    'k' : [2],
    'H' : [81],
    'r0' : [1.5],
    'R' : [57.5],
    'area' :  [math.pi * (57.5**2)],
    'rho' : [1.225],
    'lambda_opt' : [7.6],
    'Cp_max' : [0.486],
    'Bt' : [0],
    'Kt' : [0],
    'Mt' : [0],
    'eta_dt' : [0.97],
    'eta_dt2' : [0.92],
    'eta_gc' : [0.98],
    'zeta' : [0.6],
    'zeta2' : [0.45],
    'zeta3' : [0.9],
    'omega_n' : [11.11],
    'omega_n2' : [5.73],
    'omega_n3' : [3.42],
    'Bdt' : [775.49],
    'Kdt' : [2.7*1e9],
    'Br' : [7.11],
    'Bg' : [45.6],
    'Ng' : [95],
    'Jr' : [55*1e6],
    'Jg' : [390*1e6],
    'K_opt' : [1.1271],
    'K_i' : [1],
    'K_p' : [4],
    'omega_nom' : [162],
    'omega_dt' : [15],
    'Pr' : [4.8*1e6],
}

df = pd.DataFrame(params,index=None)
df.to_csv('parameters.csv',index=False)

path = 'parameters.csv'
df = pd.read_csv(path)
params = df.to_dict()

In [2]:
class Dynamics:
    def __init__(self, **params):
        self.pi = ['pi']
        self.alpha = ['alpha']
        self.alpha_gc = ['alpha_gc']
        self.a = ['a']
        self.k = ['k']
        self.H = ['H']
        self.r0 = ['r0']
        self.R = ['R']
        self.area = ['area']
        self.rho = ['rho']
        self.lambda_opt = ['lambda_opt']
        self.Cp_max = ['Cp_max']
        self.Bt = ['Bt']
        self.Kt = ['Kt']
        self.Mt = ['Mt']
        self.eta_dt = ['eta_dt']
        self.eta_dt2 = ['eta_dt2']
        self.eta_gc = ['eta_gc']
        self.zeta = ['zeta']
        self.zeta2 = ['zeta2']
        self.zeta3 = ['zeta3']
        self.omega_n = ['omega_n']
        self.omega_n2 = ['omega_n2']
        self.omega_n3 = ['omega_n3']
        self.Bdt = ['Bdt']
        self.Kdt = ['Kdt']
        self.Br = ['Br']
        self.Bg = ['Bg']
        self.Ng = ['Ng']
        self.Jr = ['Jr']
        self.Jg = ['Jg']
        self.K_opt = ['K_opt']
        self.K_i = ['K_i']
        self.K_p = ['K_p']
        self.omega_nom = ['omega_nom']
        self.omega_dt = ['omega_dt']
        self.Pr = ['Pr']

    def dynamics(self, t, X, U):
        

        rho = self.rho
        pi = self.pi
        R = self.R
        Bdt = self.Bdt 
        Kdt = self.Kdt 
        Jr = self.Jr 
        Jg = self.Jg 
        Bg = self.Bg
        Ng = self.Ng
        eta_dt = self.eta_dt
        Kt = self.Kt
        Mt = self.Mt
        zeta = self.zeta
        omega_n = self.omega_n

        tau_r = 

       
    
    def compute_input(self, v, X, uk=0):
        """
        Compute control input u as u = Kx + uf, where
        uf is a feedforward term.
        
        Parameters:
        x: numpy array
            Current state [x, y, theta, vx, vy, w]
        K: numpu array matrix-alike
            Gain matrix.
        """
        
        wr = X[0]
        wg = X[1]
        rho = self.rho
        A = self.A
        R = self.R
        lmbd_star = self.lmbd_opt + uk
        lmbd = wr*R/v
        Cp_star = self.Cp_star
        if lmbd_star != self.lmbd_opt:
            Cp_star = Cp_calc(lmbd_star)
        Cp = Cp_calc(lmbd)
        self.cp_.append(Cp_star)
        self.lmbd_.append(lmbd_star)
        K_mppt = 0.5 * rho * A * (R**3) * Cp_star/(lmbd_star**3)
        #print(lmbd_star,Cp_star,K_mppt)
        tR = 0.5*rho*A*Cp*(v**3)/wr
        tG = K_mppt*(wg**2)
        du = np.array([tR,tG])
        self.U = du
        self.K_mppt = np.append(self.K_mppt,K_mppt)
        
        return du
    
    def iterate(self, u, dt):
        """
        Propagate the system forward in time by 'dt' using 
        numerical integration.
        
        Parameters:
        u : array-like
            Input vector [u1, u2]
        dt : float
            Time step
        """
        #DOP853
        sol = solve_ivp(self.dynamics, [0, dt], self.X, 
            args=(u.flatten(),), method="DOP853", t_eval=[dt])
        self.X = sol.y[:, -1]
        return self.X
    
    def compute_output(self):
        X = self.X.reshape(1,-1)
        dX = self.dX.reshape(1,-1)
        B = self.Bdt
        tau_r, tau_g = self.U
        w_r, w_g, _ = self.X
        PG = tau_g*w_g
        PD = B*((w_r-w_g)**2)

        #if self.k>=300:
        EG = self.EG[-1]+PG
        ED = self.ED[-1]+PD

        self.PD = np.append(self.PD,PD)
        self.PG = np.append(self.PG,PG) 
        self.ED = np.append(self.ED,ED)
        self.EG = np.append(self.EG,EG) 
        self.X_ = np.append(self.X_,X,axis=0)  
        self.dX_ = np.append(self.dX_,dX,axis=0)  
        if self.k == 1:
            self.PD = np.delete(self.PD,0)
            self.PG = np.delete(self.PG,0) 
            self.ED = np.delete(self.ED,0)
            self.EG = np.delete(self.EG,0) 

        self.k = self.k + 1

SyntaxError: invalid syntax (3192133561.py, line 59)

In [3]:
path = r'Datasets\CpVersusTSR&Pitch_WT_Perf_V1.csv'
df = pd.read_csv(path)
df

,TSR,-5°,-4°,-3°,-2°,-1°,0°,1°,2°,3°,4°,5°
0,0.411942,0.0007,0.0010,0.0013,0.0015,0.0018,0.0021,0.0023,0.0026,0.0028,0.0031,0.0033
1,0.439404,0.0008,0.0011,0.0014,0.0016,0.0019,0.0022,0.0025,0.0027,0.0030,0.0033,0.0035
2,0.470790,0.0009,0.0012,0.0015,0.0018,0.0021,0.0024,0.0027,0.0030,0.0032,0.0035,0.0038
3,0.507005,0.0009,0.0013,0.0016,0.0019,0.0022,0.0026,0.0029,0.0032,0.0035,0.0038,0.0041
4,0.549255,0.0010,0.0014,0.0017,0.0021,0.0024,0.0028,0.0032,0.0035,0.0038,0.0042,0.0045
...,...,...,...,...,...,...,...,...,...,...,...,...
191,23.068729,-0.5313,-0.5180,-0.5276,-0.5396,-0.5539,-0.5790,-0.6162,-0.6788,-0.8500,-1.1788,-1.7242
192,24.167240,-0.6381,-0.6257,-0.6392,-0.6585,-0.6841,-0.7269,-0.7918,-0.8824,-1.0819,-1.4697,-2.1104
193,26.364261,-0.8841,-0.8743,-0.8973,-0.9319,-0.9832,-1.0642,-1.1895,-1.3443,-1.6264,-2.1537,-3.0514
194,28.561283,-1.1772,-1.1713,-1.2051,-1.2564,-1.3369,-1.4665,-1.6582,-1.9132,-2.2992,-3.0409,-4.1698


In [4]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Assuming df is already loaded as a pandas DataFrame
X = df.iloc[:, 0].values
Y = np.arange(len(df.columns)-1)
X, Y = np.meshgrid(X, Y)

Z = np.array(df.iloc[:, 1:]).T

# Create the plot
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='magma')])

# Customize layout
fig.update_layout(
    scene = {
        'xaxis_title': 'X Axis',
        'yaxis_title': 'Y Axis',
        'zaxis_title': 'Z Axis',
        #'zaxis': dict(range=[-2.01, 0.5])
    },
    coloraxis_colorbar=dict(
        title="Color Scale"
    ),
)

# Show the plot
fig.show()


In [7]:
import numpy as np
from scipy.signal import tf2ss

# Assume omega_n and xi are defined, for example:
omega_n = 10.0
xi = 0.7

# Define the numerator and denominator
num = [omega_n**2]
den = [1, 2*xi*omega_n, omega_n**2]

# Perform the conversion
Apb, Bpb, Cpb, Dpb = tf2ss(num, den)

print("A matrix:\n", Apb)
print("\nB matrix:\n", Bpb)
print("\nC matrix:\n", Cpb)
print("\nD matrix:\n", Dpb)

A matrix:
 [[ -14. -100.]
 [   1.    0.]]

B matrix:
 [[1.]
 [0.]]

C matrix:
 [[  0. 100.]]

D matrix:
 [[0.]]
